In [1]:
import pandas as pd
import numpy as np

import os
import random
import cv2

import numpy as np
from wordcloud import WordCloud, STOPWORDS
import seaborn as sns
from os import path
sns.set()
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score
import datetime as dt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import calendar
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
%matplotlib inline
import time
# !pip install tensorflow
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.layers import Dense, Dropout

from keras.layers import Conv1D, Flatten,MaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.metrics import auc

# Regular Expression for text cleaning
import re
# to track the progress - progress bar
from tqdm.notebook import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Load already cleaned text.

In [16]:
df = pd.read_csv("/content/drive/My Drive/Kaggle Club/SARCASM PROJECT '25/cleaned_reddit_comments.csv").fillna(' ')
df

,label,comment
0,0,nc and nh.
1,0,you do know west teams play against west teams...
2,0,"they were underdogs earlier today , but since ..."
3,0,"this meme is not funny none of the "" new york ..."
4,0,i could use one of those tools.
...,...,...
1010821,1,i am sure that iran and n. korea have the tech...
1010822,1,"whatever you do , do not vote green !"
1010823,1,perhaps this is an atheist conspiracy to make ...
1010824,1,the slavs got their own country - it is called...


# LSTM-specific Preprocessing

In [17]:
# Lemmatize
import spacy
nlp = spacy.load("en_core_web_sm")

def lemmatize_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc if not token.is_punct])  # Remove punctuation

# (FOR TESTING SAMPLES)
# sample_comments = df["comment"].head(5)
# sample_comments_lemmatized = sample_comments.apply(lemmatize_text)
# for original, lemmatized in zip(sample_comments, sample_comments_lemmatized):
#     print(f"Original: {original}\nLemmatized: {lemmatized}\n")

df['comment'] = df['comment'].apply(lemmatize_text)
df


,label,comment
0,0,nc and nh
1,0,you do know west team play against west team m...
2,0,they be underdog early today but since gronk s...
3,0,this meme be not funny none of the new york ni...
4,0,I could use one of those tool
...,...,...
1010821,1,I be sure that iran and n. korea have the tech...
1010822,1,whatever you do do not vote green
1010823,1,perhaps this be an atheist conspiracy to make ...
1010824,1,the slavs get their own country it be call kosovo


In [18]:
# Remove stop words (commonly used words that can act as noise)
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return " ".join(filtered_words)

df['comment'] = df['comment'].apply(remove_stopwords)
df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,label,comment
0,0,nc nh
1,0,know west team play west team east team right
2,0,underdog early today since gronk announcement ...
3,0,meme funny none new york nigga one
4,0,could use one tool
...,...,...
1010821,1,sure iran n. korea technology create pig bird ...
1010822,1,whatever vote green
1010823,1,perhaps atheist conspiracy make christian look...
1010824,1,slavs get country call kosovo


In [19]:
# Save dataset for LSTM
output_path = "/content/drive/My Drive/Kaggle Club/SARCASM PROJECT '25/cleaned_reddit_comments_LSTM.csv"
df.to_csv(output_path, index=False)